In [ ]:
# !rm -rf "/kaggle/working/unet"

# #  xóa thư mục nếu sai (dùng để test n lần model tràn ram)

In [1]:
!pip install transformers safetensors onnx onnxruntime
!pip uninstall -y jax jaxlib
!pip install --no-cache-dir -U jax jaxlib
!pip install --no-cache-dir -U transformers diffusers torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118

Found existing installation: jax 0.5.2
Uninstalling jax-0.5.2:
  Successfully uninstalled jax-0.5.2
Found existing installation: jaxlib 0.5.1
Uninstalling jaxlib-0.5.1:
  Successfully uninstalled jaxlib-0.5.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 38.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 MB 272.8 MB/s eta 0:00:00a 0:00:01
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118


In [2]:
!pip uninstall -y torch torchvision torchaudio
!pip install --no-cache-dir torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Found existing installation: torch 2.6.0+cu118
Uninstalling torch-2.6.0+cu118:
  Successfully uninstalled torch-2.6.0+cu118
Found existing installation: torchvision 0.21.0+cu118
Uninstalling torchvision-0.21.0+cu118:
  Successfully uninstalled torchvision-0.21.0+cu118
Found existing installation: torchaudio 2.6.0+cu118
Uninstalling torchaudio-2.6.0+cu118:
  Successfully uninstalled torchaudio-2.6.0+cu118
Looking in indexes: https://download.pytorch.org/whl/cu118
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 848.7/848.7 MB 264.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 278.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 260.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.18 requires torch<2.6,>=1.10, but you have torch 2.6.0+cu118 which is incompatible.


In [4]:
import os
dataset_path = "/kaggle/input/pix-2-onnx"
if os.path.exists(dataset_path):
    print(f"Dataset found at: {dataset_path}")
    print("Files:", os.listdir(dataset_path))
else:
    print("Dataset not found. Make sure it is attached to the notebook.")

Dataset found at: /kaggle/input/pix-2-onnx
Files: ['tokenizer', 'instruct-pix2pix-00-22000.safetensors', 'scheduler', 'vae', 'text_encoder', 'model_index.json', 'feature_extractor', 'unet', 'instruct-pix2pix-00-22000.ckpt']


*TEXT_ENCODER*

In [5]:
import os
import torch
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import UNet2DConditionModel, StableDiffusionInstructPix2PixPipeline

# Đảm bảo dataset đã được gắn vào notebook trên Kaggle
dataset_path = "/kaggle/input/pix-2-onnx"
if not os.path.exists(dataset_path):
    raise FileNotFoundError("Dataset 'pix-2-onnx' không được tìm thấy. Hãy gắn dataset vào notebook.")

# Load tokenizer từ thư mục phù hợp
tokenizer = CLIPTokenizer.from_pretrained("timbrooks/instruct-pix2pix", subfolder="tokenizer")

# Load text encoder từ thư mục phù hợp
text_encoder = CLIPTextModel.from_pretrained("timbrooks/instruct-pix2pix", subfolder="text_encoder").eval()

# Dummy input cho ONNX export
dummy_input = torch.randint(0, 49408, (1, 77))  # Tokenized text input

# Xuất sang ONNX
torch.onnx.export(
    text_encoder,
    dummy_input,
    "/kaggle/working/text_encoder.onnx",
    input_names=["input_text"],
    output_names=["text_embeddings"],
    dynamic_axes={"input_text": {0: "batch_size"}, "text_embeddings": {0: "batch_size"}},
    opset_version=17
)

print("Text Encoder ONNX exported successfully!")



tokenizer_config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/clip/modeling_clip.py:282: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if seq_length > max_position_embedding:
/usr/local/lib/python3.10/dist-packages/transformers/modeling_attn_mask_utils.py:88: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if input_shape[-1] > 1 or self.sliding_window is not None:
/usr/local/lib/python3.10/dist-packages/transformers/modeling_attn_mask_utils.py:164: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record

Text Encoder ONNX exported successfully!


In [6]:
##check model
import os
print(os.getcwd())  
print(os.listdir()) 


/kaggle/working
['text_encoder.onnx', '.virtual_documents']


In [7]:
onnx_path = "/kaggle/working/text_encoder.onnx" 
print(os.path.exists(onnx_path))  

True


*UNET*

In [8]:
import torch
from diffusers import UNet2DConditionModel, StableDiffusionInstructPix2PixPipeline

# Load the pipeline
model_id = "timbrooks/instruct-pix2pix"
pipe = StableDiffusionInstructPix2PixPipeline.from_pretrained(
    model_id, torch_dtype=torch.float16, safety_checker=None
)
unet = pipe.unet.to("cuda").eval()

# Define correct data type
dtype = torch.float16  # Ensure all inputs match model's dtype

# Dummy inputs (must be float16)
batch_size = 1
latent_channels = 4
latent_height = 64
latent_width = 64

# Convert all inputs to float16
dummy_noise_latents = torch.randn(batch_size, latent_channels, latent_height, latent_width, device="cuda", dtype=dtype)
dummy_image_latents = torch.randn(batch_size, latent_channels, latent_height, latent_width, device="cuda", dtype=dtype)
dummy_latents = torch.cat([dummy_noise_latents, dummy_image_latents], dim=1)

dummy_timestep = torch.tensor([1], device="cuda", dtype=dtype)
dummy_text_embeddings = torch.randn(batch_size, 77, 768, device="cuda", dtype=dtype)

# Path to save ONNX model in Kaggle environment
save_path = "/kaggle/working/unet.onnx"

# Export to ONNX
torch.onnx.export(
    unet,
    (dummy_latents, dummy_timestep, dummy_text_embeddings),
    save_path,
    input_names=["latents", "timestep", "text_embeddings"],
    output_names=["predicted_noise"],
    dynamic_axes={"latents": {0: "batch_size"}, "text_embeddings": {0: "batch_size"}},
    opset_version=17
)

print(f"UNet ONNX exported successfully! Saved at: {save_path}")


model_index.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/518 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

scheduler_config.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/diffusers/models/unets/unet_2d_condition.py:1111: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if dim % default_overall_up_factor != 0:
/usr/local/lib/python3.10/dist-packages/diffusers/models/downsampling.py:136: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert hidden_states.shape[1] == self.channels
/usr/local/lib/python3.10/dist-packages/diffusers/models/downsampling.py:145: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Pytho

UNet ONNX exported successfully! Saved at: /kaggle/working/unet.onnx


*VAE_ENCODER* Chuyển ảnh thành latents.

In [10]:
import torch
from diffusers import AutoencoderKL

# Load VAE từ instruct-pix2pix
vae = AutoencoderKL.from_pretrained("timbrooks/instruct-pix2pix", subfolder="vae").to("cuda").eval()

# Dummy input (ảnh 3 kênh RGB 512x512)
dummy_image = torch.randn(1, 3, 512, 512, device="cuda", dtype=torch.float32)

# Lấy encoder model
vae_encoder = vae.encoder

# Export VAE Encoder
torch.onnx.export(
    vae_encoder,
    dummy_image,
    "vae_encoder.onnx",
    input_names=["image"],
    output_names=["latent"],
    dynamic_axes={"image": {0: "batch_size"}, "latent": {0: "batch_size"}},
    opset_version=17
)

print("VAE Encoder ONNX exported successfully!")


/usr/local/lib/python3.10/dist-packages/torch/onnx/_internal/jit_utils.py:308: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at /pytorch/torch/csrc/jit/passes/onnx/constant_fold.cpp:178.)
  _C._jit_pass_onnx_node_shape_type_inference(node, params_dict, opset_version)
/usr/local/lib/python3.10/dist-packages/torch/onnx/utils.py:657: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at /pytorch/torch/csrc/jit/passes/onnx/constant_fold.cpp:178.)
  _C._jit_pass_onnx_graph_shape_type_inference(
/usr/local/lib/python3.10/dist-packages/torch/onnx/utils.py:1127: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at /pytorch/torch/csrc/jit/passes/onnx/constant_fold.cpp:178.)
  _C._jit_pass_on

VAE Encoder ONNX exported successfully!


*VAE_DECODER* Chuyển latents thành ảnh.

In [12]:
# Lấy decoder model
vae_decoder = vae.decoder

# Dummy latent input (4 channels, 64x64)
dummy_latent = torch.randn(1, 4, 64, 64, device="cuda", dtype=torch.float32)

# Export VAE Decoder
torch.onnx.export(
    vae_decoder,
    dummy_latent,
    "vae_decoder.onnx",
    input_names=["latent"],
    output_names=["image"],
    dynamic_axes={"latent": {0: "batch_size"}, "image": {0: "batch_size"}},
    opset_version=17
)

print("VAE Decoder ONNX exported successfully!")


VAE Decoder ONNX exported successfully!
